In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.basemap import Basemap
import networkx as nx
import networkx.algorithms.community as nxcom
import networkx.algorithms.approximation as nxapp
import community as community_louvain
from igraph import *
#from cdlib import algorithms

In [ ]:
# read stored data from data_preparation notebook
%store -r df_airports
%store -r airports_dict
%store -r df_merged

# Community Detection

In [ ]:
# load network (for networkx and igraph)
G = nx.read_gml('Graphs/airlines.gml').to_undirected()
#g = Graph.Read_GML('Graphs/airlines.gml').as_undirected()

In [ ]:
# compute assortativity
assortativity = nx.degree_pearson_correlation_coefficient(G, weight=None)
assortativity

A community is a group of system constituents who share
common properties and/or play similar roles within the
system. Nodes within the same community tend to be connected
with each other much more frequently within the community
than with nodes outside

We explore comminities withing the airline network with the following
modularity based, random walk an other community detection algorithms:
- Fast greedy (modularity based)
- Multilevel (modularity based)
- Walktrap (random walk)
- Edge betweenness
- Label propagation

**Note:** The lecture mentions another algorithm, Leading eigenvector, which we
will not further explore since it was mentioned that it is neither stable nor
fast to compute.

**Note:** The difficulty of this task is that we do not know the "correct" communities, therefore
we cannot easily compare the performance of the algorithms. What we can do is to compare it with
(a.) geographical location and (b.) with the resulting community detection performed by the
reference paper. We can also reason about the actual computational performance of each
algorithm and its stability which we can use for the discussion of the final report.

Networkx Reference: https://networkx.org/documentation/stable/reference/algorithms/community.html

In [ ]:
# compute random vertex color used for commmunity visualisation
memory = {}
def random_color(number, memory):
    if not number in memory:
        r = random.random()
        g = random.random()
        b = random.random()
        memory[number] = (r, g, b, 1.0)
    return memory[number]

# draw community graphs
def draw_community_graph(g, communities, with_labels=True):
    plt.figure(figsize=(8,8))
    # add community attribute
    for set_idx, frozenset in enumerate(communities):
        for node_idx in frozenset:
            g.nodes[node_idx]['community'] = set_idx
    # color mapping
    color_map = [random_color(g.nodes[v]['community'], memory) for v in g.nodes]
    nx.draw(g, with_labels=with_labels, node_color=color_map)
    plt.show()

## Fast greedy Algorithm

Bottom up hierarchical decomposition process. It will merge two current communities
iteratively, with the goal to achieve the maximum modularity gain at local optimal.

In [ ]:
communities_greedy = sorted(nxcom.greedy_modularity_communities(G), key=len, reverse=True)
draw_community_graph(G, communities_greedy, with_labels=False)
print(f'Fast greedy algorithm resulted in {len(communities_greedy)} communities')

In [ ]:
lats = [k[1]['Latitude'] for k in airports_dict.items()]
longs = [k[1]['Longitude'] for k in airports_dict.items()]

In [ ]:
plt.figure(figsize=(20, 20))
map = Basemap(projection='cyl', resolution=None,
              llcrnrlat=-90, urcrnrlat=90,
              llcrnrlon=-180, urcrnrlon=180, )

pos = {}
for n in G.nodes():
    x,y = map(G.nodes[n]['Longitude'], G.nodes[n]['Latitude'])
    pos[n] = (x,y)

# add community attribute
for set_idx, frozenset in enumerate(communities_greedy):
    for node_idx in frozenset:
        G.nodes[node_idx]['community'] = set_idx
# color mapping
color_map = [random_color(G.nodes[v]['community'], memory) for v in G.nodes]
# draw map
map.shadedrelief()
nx.draw_networkx(G, pos=pos, node_size=50, edge_color='r', node_color=color_map,
                 arrows=False, with_labels=False, alpha=1, width=00, node_shape='.')
plt.title('Air Traffic Communities (Greedy Modularity Algorithm)')
plt.savefig('Figures/communities/gma.pdf')
plt.show()

## Multilevel Algorithm

**Idea:** Merge smaller partitions into larger ones as long as the modularity score improves.

In [ ]:
plt.figure(figsize=(8, 6))
communities = community_louvain.best_partition(G)
 # draw the graph
pos = nx.spring_layout(G)
# color the nodes according to their partition
cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
nx.draw_networkx_nodes(G, pos, partition.keys(), cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()

In [ ]:
communities_ml = g.community_multilevel()
# create nodelist with its community (to be zipped)
nodelist = []
communitylist = []
# fill dict
comm_idx = 0
for c in communities_ml:
    for e in c:
        nodelist.append(e)
        communitylist.append(c)
    comm_idx +=1

In [ ]:
# attach community information
for n,c in zip(nodelist, communitylist):
    try:
        G.nodes[n]['community'] = c
    except KeyError:
        pass # ignore?

In [ ]:
#for x in G.nodes:
#    print(G.nodes[x]['community'])
G.nodes[2369]['community']

In [ ]:
color_map = [get_color(G.nodes[v]['community']) for v in G.nodes]
nx.draw(G, with_labels=False, node_color=color_map)
plt.show()

In [ ]:
plot(communities_ml, mark_groups = True)

In [ ]:
patritions = sorted(community_louvain.best_partition(G), key=len, reverse=True)
draw_community_graph(G, patritions)

## Walktrap Algorithm

https://cdlib.readthedocs.io/en/latest/reference/cd_algorithms/algs/cdlib.algorithms.walktrap.html

**Idea:** Short-distance random walk tends to stay in the same community.

In [ ]:
communities_wt = g.community_walktrap()
plot(communities_wt, mark_groups = True)

## Edge Betweenness Algorithm

**Idea:** Detect communities by progressively removing edges with the highest edge
 betweenness centrality from the original network

In [ ]:
communities_eb = sorted(nxcom.girvan_newman(G), key=len, reverse=True)
draw_community_graph(G, communities_eb, with_labels=False)
print(f'Edge betweenness algorithm resulted in {len(communities_eb)} communities')

In [ ]:
plt.figure(figsize=(20, 20))
map = Basemap(projection='cyl', resolution=None,
              llcrnrlat=-90, urcrnrlat=90,
              llcrnrlon=-180, urcrnrlon=180, )

pos = {}
for n in G.nodes():
    x,y = map(G.nodes[n]['Longitude'], G.nodes[n]['Latitude'])
    pos[n] = (x,y)

# add community attribute
for set_idx, frozenset in enumerate(communities_eb):
    for node_idx in frozenset:
        G.nodes[node_idx]['community'] = set_idx
# color mapping
color_map = [random_color(G.nodes[v]['community'], memory) for v in G.nodes]
# draw map
map.shadedrelief()
nx.draw_networkx(G, pos=pos, node_size=50, edge_color='r', node_color=color_map,
                 arrows=False, with_labels=False, alpha=1, width=00, node_shape='.')
plt.title('Air Traffic Communities (Greedy Modularity Algorithm)')
plt.savefig('Figures/communities/eba.pdf')
plt.show()

## Label Propagation Algorithm

**Idea:** Assign each node in the network to the community to which belongs the majority
of its neighbours.

In [ ]:

communities_lp = sorted(nxcom.label_propagation_communities(G), key=len, reverse=True)
draw_community_graph(G, communities_lp, with_labels=False)
print(f'Label propagation algorithm resulted in {len(communities_lp)} communities')

In [ ]:
plt.figure(figsize=(20, 20))
map = Basemap(projection='cyl', resolution=None,
              llcrnrlat=-90, urcrnrlat=90,
              llcrnrlon=-180, urcrnrlon=180, )

pos = {}
for n in G.nodes():
    x,y = map(G.nodes[n]['Longitude'], G.nodes[n]['Latitude'])
    pos[n] = (x,y)

# add community attribute
for set_idx, frozenset in enumerate(communities_lp):
    for node_idx in frozenset:
        G.nodes[node_idx]['community'] = set_idx
# color mapping
color_map = [random_color(G.nodes[v]['community'], memory) for v in G.nodes]
# draw map
map.shadedrelief()
nx.draw_networkx(G, pos=pos, node_size=50, edge_color='r', node_color=color_map,
                 arrows=False, with_labels=False, alpha=1, width=00, node_shape='.')
plt.title('Air Traffic Communities (Greedy Modularity Algorithm)')
plt.savefig('Figures/communities/lpa.pdf')
plt.show()

# Simulated annealing (used in reference paper)

see:
- https://networkx.org/documentation/stable//reference/algorithms/generated/networkx.algorithms.approximation.traveling_salesman.simulated_annealing_tsp.html
- https://en.wikipedia.org/wiki/Simulated_annealing

In [ ]:
#sim_ann = nxapp.simulated_annealing_tsp(G, init_cycle="greedy")